# Calculate PSL and save as a file
RUFMOD ONLY

* **Description**: Reads in and creates seasonal and ensemble means and vertically interpolates
* **Input data**: Rufmod output in timeseries format
* **Output data**: Netcdf file with output
* **Creator**: Alice DuVivier
* **Date**: April 2022

The rufmod experiments were performed where the sea ice roughness over Arctic sea ice regions was set to be equal to what it would be over open ocean. This is to better understand ice-atmosphere coupling, processes, and feedbacks.

In [1]:
import xarray as xr
import numpy as np
from datetime import timedelta
import glob

import pop_tools

import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.gridspec import GridSpec

import geocat.datafiles as gdf
import geocat.viz.util as gvutil
from geocat.viz import cmaps as gvcmaps
import geocat.comp as gcomp

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.stats import linregress,pearsonr, t

import dask
import intake
from distributed import Client
from ncar_jobqueue import NCARCluster

/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
# spin up dask cluster

import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
cluster = PBSCluster(
    cores=36, # The number of cores you want
    memory='300 GB', # Amount of memory
    processes=9, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus=36:mem=300GB', # Specify resources
    project='P93300665', # Input your project ID here
    walltime='00:30:00', # Amount of wall time
    interface='ib0', # Interface to use
)
# Scale up
cluster.scale(jobs=4)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.42:39976,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Manually set variables

In [4]:
# list the variables to load
var_in_1 = 'PSL'

## Load rufmod experiments

In [5]:
# Load "rufmod" data
#choose cases and data paths
case1 = 'b.e21.BSSP370.f09_g17.rufmod.001'
case2 = 'b.e21.BSSP370.f09_g17.rufmod.002'
case3 = 'b.e21.BSSP370.f09_g17.rufmod.003'
case4 = 'b.e21.BSSP370.f09_g17.rufmod.004'
case5 = 'b.e21.BSSP370.f09_g17.rufmod.005'

# set base directory where all data live
data_dir = '/glade/campaign/cesm/development/pcwg/projects/arctic_cyclones/rufmod_expts/'
# set individual data directories
data_dir1 = data_dir+case1+'/atm/proc/tseries/month_1/'
data_dir2 = data_dir+case2+'/atm/proc/tseries/month_1/'
data_dir3 = data_dir+case3+'/atm/proc/tseries/month_1/'
data_dir4 = data_dir+case4+'/atm/proc/tseries/month_1/'
data_dir5 = data_dir+case5+'/atm/proc/tseries/month_1/'

In [6]:
%%time
#reading in files
print("loading "+var_in_1)   
ds1_1 = []
ds2_1 = []
ds3_1 = []
ds4_1 = []
ds5_1 = []
my_files=sorted(glob.glob(data_dir1+case1+'.cam.h0.'+var_in_1+'.*.nc'))
ds1_1=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir2+case2+'.cam.h0.'+var_in_1+'.*.nc'))
ds2_1=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir3+case3+'.cam.h0.'+var_in_1+'.*.nc'))
ds3_1=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')    
my_files=sorted(glob.glob(data_dir4+case4+'.cam.h0.'+var_in_1+'.*.nc'))
ds4_1=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')
my_files=sorted(glob.glob(data_dir5+case5+'.cam.h0.'+var_in_1+'.*.nc'))
ds5_1=xr.open_mfdataset(my_files,combine='by_coords',chunks={'time':129}, parallel=True, compat='override', coords='minimal')


loading PSL
CPU times: user 1.15 s, sys: 418 ms, total: 1.57 s
Wall time: 39.4 s


In [7]:
# concatenate them into a single array
futures_1 = xr.concat([ds1_1,ds2_1,ds3_1,ds4_1,ds5_1],dim='member_id')

In [9]:
futures_1.PSL

<xarray.DataArray 'PSL' (member_id: 5, time: 1032, lat: 192, lon: 288)>
dask.array<concatenate, shape=(5, 1032, 192, 288), dtype=float32, chunksize=(1, 129, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 2015-02-01 00:00:00 ... 2101-01-01 00:00:00
Dimensions without coordinates: member_id
Attributes:
    units:         Pa
    long_name:     Sea level pressure
    cell_methods:  time: mean

In [10]:
# set member_id values
futures_1.member_id.values

# assign member_id as coordinate array
futures_1 = futures_1.assign_coords({"member_id": futures_1.member_id.values})

In [11]:
# Shift months by one to be center of time period.
# Take average of the time bounds to get middle of month
# will lose some attributes with time, so may need to put this back in later...
futures_1['time'] = futures_1.time_bnds.load().mean(dim='nbnd').sel(member_id=0)

In [12]:
# get just NH slice
futures_1_masked = futures_1.isel(lat=slice(164,192))

In [13]:
# grab variables of interest
PSL_rufmod = futures_1_masked[var_in_1]

# Actually load data

In [14]:
PSL_rufmod.load()

<xarray.DataArray 'PSL' (member_id: 5, time: 1032, lat: 28, lon: 288)>
array([[[[ 98824.69 ,  98853.04 ,  98889.445, ...,  98794.41 ,
           98800.3  ,  98807.4  ],
         [ 98790.01 ,  98808.42 ,  98835.12 , ...,  98771.73 ,
           98774.75 ,  98781.46 ],
         [ 98774.77 ,  98781.445,  98799.03 , ...,  98777.29 ,
           98773.266,  98773.96 ],
         ...,
         [ 99999.14 ,  99992.45 ,  99985.64 , ..., 100018.88 ,
          100012.27 , 100005.73 ],
         [ 99997.875,  99995.36 ,  99992.84 , ..., 100005.51 ,
          100002.87 , 100000.33 ],
         [100015.98 , 100015.98 , 100015.98 , ..., 100015.98 ,
          100015.98 , 100015.98 ]],

        [[ 98989.125,  98995.5  ,  99004.77 , ...,  98981.266,
           98980.17 ,  98983.4  ],
         [ 98958.36 ,  98953.92 ,  98954.35 , ...,  98990.3  ,
           98975.47 ,  98964.836],
         [ 98904.24 ,  98890.09 ,  98883.24 , ...,  98976.8  ,
           98949.81 ,  98924.49 ],
...
         [101269.78 , 101269.6  , 101269.555, ..., 101270.33 ,
          101270.06 , 101269.9  ],
         [101365.336, 101365.125, 101365.01 , ..., 101366.13 ,
          101365.836, 101365.56 ],
         [101464.13 , 101464.13 , 101464.13 , ..., 101464.13 ,
          101464.13 , 101464.13 ]],

        [[100734.16 , 100750.25 , 100769.   , ..., 100704.016,
          100711.94 , 100724.28 ],
         [100651.79 , 100668.79 , 100685.586, ..., 100616.79 ,
          100626.76 , 100640.49 ],
         [100571.57 , 100588.41 , 100604.66 , ..., 100531.62 ,
          100541.69 , 100556.414],
         ...,
         [100758.2  , 100756.85 , 100755.53 , ..., 100762.555,
          100760.93 , 100759.43 ],
         [100776.98 , 100776.15 , 100775.38 , ..., 100779.695,
          100778.72 , 100777.82 ],
         [100798.45 , 100798.45 , 100798.45 , ..., 100798.45 ,
          100798.45 , 100798.45 ]]]], dtype=float32)
Coordinates:
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
  * member_id  (member_id) int64 0 1 2 3 4
Attributes:
    units:         Pa
    long_name:     Sea level pressure
    cell_methods:  time: mean

## Calculate Seasonal Means

In [16]:
season_names = ['OND','JFM', 'AMJ', 'JAS']

In [17]:
# find total years
xarr_rufmod = PSL_rufmod.coords['time.year'][(PSL_rufmod.coords['time.month']==1)]

In [19]:
# Loop through seasons - rufmod

# make numpy array to fill and specify dimensions we want
seas_array_rufmod_1 = np.zeros([len(season_names),len(xarr_rufmod),len(PSL_rufmod.member_id),len(PSL_rufmod.lat),len(PSL_rufmod.lon)])

for s_count, ss in enumerate(season_names):
    print(ss)
    ### Z PLEV
    # get temporary array of just these month by season
    if ss == 'JFM':
        temp1 = PSL_rufmod.isel(time=PSL_rufmod.time.dt.month.isin([1,2,3]))
    if ss == 'AMJ':
        temp1 = PSL_rufmod.isel(time=PSL_rufmod.time.dt.month.isin([4,5,6]))
    if ss == 'JAS':
        temp1 = PSL_rufmod.isel(time=PSL_rufmod.time.dt.month.isin([7,8,9]))
    if ss == 'OND':
        temp1 = PSL_rufmod.isel(time=PSL_rufmod.time.dt.month.isin([10,11,12]))
    # now loop through years to get the seasonal average by year for each ensemble member
    for y_count, yy in enumerate(xarr_rufmod):
        # select only the indexes for this year
        temp1a = temp1.isel(time=temp1.time.dt.year.isin([yy])).mean(dim='time')
        seas_array_rufmod_1[s_count,y_count,:,:,:] = temp1a 
        

OND
JFM
AMJ
JAS


In [20]:
print(seas_array_rufmod_1.shape)

(4, 86, 5, 28, 288)


In [21]:
# convert the numpy array to a xarray for easier plotting
PSL_seas_rufmod = xr.DataArray(seas_array_rufmod_1,dims=('season','time','member_id','lat','lon'))

In [22]:
# set coordinate arrays
PSL_seas_rufmod['season'] = season_names
PSL_seas_rufmod['time'] = xarr_rufmod
PSL_seas_rufmod['member_id'] = PSL_rufmod['member_id']
PSL_seas_rufmod['lat'] = PSL_rufmod['lat'].values
PSL_seas_rufmod['lon'] = PSL_rufmod['lon'].values

## Write out files

In [23]:
# quick and dirty way to save a file!

# save rufmod expt, rename the variable so it makes sense
#fout = 'rufmod_vertical_seas_ens_mean_WS'
#
#WS_seas_ens_mean_rufmod.to_dataset(name='vert_ws').to_netcdf(fout+'.nc')

### PSL

In [29]:
#set info to write out
out_tag = 'PSL'
units = 'Pa'
longname = 'sea level pressure'

fout = 'rufmod_seas_'+out_tag

In [30]:
ds_to_save = PSL_seas_rufmod

In [31]:
# check how big this will be to write out in GB
ds_to_save.nbytes/(1024**3)

0.10334014892578125

In [32]:
# assign some attributes
refdata = {'Author': 'Alice DuVivier', 'units':units, 'longname':longname}

ds_to_save.attrs = refdata

In [33]:
# check data
ds_to_save


<xarray.DataArray (season: 4, time: 86, member_id: 5, lat: 28, lon: 288)>
array([[[[[100392.1875   , 100424.2890625, 100454.2890625, ...,
           100304.4921875, 100333.75     , 100362.4375   ],
          [100371.6796875, 100392.34375  , 100413.15625  , ...,
           100310.3515625, 100331.7109375, 100352.0390625],
          [100366.1328125, 100376.5390625, 100386.8359375, ...,
           100328.5078125, 100341.8984375, 100354.4140625],
          ...,
          [100933.1796875, 100933.3828125, 100933.5234375, ...,
           100932.5703125, 100932.7421875, 100932.9375   ],
          [100945.6015625, 100945.7890625, 100946.0859375, ...,
           100945.03125  , 100945.2578125, 100945.4609375],
          [100958.5625   , 100958.5625   , 100958.5625   , ...,
           100958.5625   , 100958.5625   , 100958.5625   ]],

         [[ 99705.5390625,  99755.       ,  99806.7578125, ...,
            99564.3984375,  99607.4296875,  99655.1875   ],
          [ 99638.0625   ,  99679.8125   ,  99723.6640625, ...,
            99521.375    ,  99556.2890625,  99596.8359375],
          [ 99604.53125  ,  99638.53125  ,  99676.4921875, ...,
            99511.3828125,  99539.71875  ,  99571.6015625],
...
          [101091.4453125, 101091.4609375, 101091.53125  , ...,
           101091.9609375, 101091.75     , 101091.5546875],
          [101135.1484375, 101134.75     , 101134.4140625, ...,
           101136.40625  , 101135.9296875, 101135.5390625],
          [101182.9140625, 101182.9140625, 101182.9140625, ...,
           101182.9140625, 101182.9140625, 101182.9140625]],

         [[101265.1171875, 101284.3203125, 101301.9609375, ...,
           101199.75     , 101221.9140625, 101244.4453125],
          [101190.875    , 101211.25     , 101229.75     , ...,
           101125.8359375, 101147.59375  , 101169.6875   ],
          [101127.7578125, 101147.7578125, 101166.4765625, ...,
           101066.375    , 101087.4765625, 101107.0390625],
          ...,
          [100759.1953125, 100760.625    , 100762.0390625, ...,
           100754.5234375, 100756.1171875, 100757.6875   ],
          [100711.4140625, 100712.0234375, 100712.6171875, ...,
           100709.3125   , 100710.0546875, 100710.78125  ],
          [100648.8515625, 100648.8515625, 100648.8515625, ...,
           100648.8515625, 100648.8515625, 100648.8515625]]]]])
Coordinates:
  * season     (season) <U3 'OND' 'JFM' 'AMJ' 'JAS'
  * time       (time) int64 2015 2016 2017 2018 2019 ... 2097 2098 2099 2100
  * member_id  (member_id) int64 0 1 2 3 4
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    Author:    Alice DuVivier
    units:     Pa
    longname:  sea level pressure

In [34]:
ds_to_save.to_netcdf(fout+'.nc')  # how to save file